In [1]:
'''
IF RECOMPILING: DELETE author_tokens.txt
''' 

'\nIF RECOMPILING: DELETE author_tokens.txt\n'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import glob
import pandas as pd
import nltk
from nltk import word_tokenize
import string
import json
import re
import os
import csv

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
!apt-get update

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [83.2 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [130 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [753 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Package

In [5]:
import os    
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null 
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  
  #Force max heap size
  os.environ["JAVA_OPTIONS"]= '-Xmx4096m' 
  #java.lang.OutOfMemoryError: Java heap space
  #times out after about 10 min
  #java.lang.OutOfMemoryError: GC overhead limit exceeded
  
  !java -version      
install_java()

#Taken with modificaiton from: 
#https://stackoverflow.com/questions/51287258/how-can-i-use-java-in-google-colab

openjdk version "1.8.0_191"
OpenJDK Runtime Environment (build 1.8.0_191-8u191-b12-0ubuntu0.18.04.1-b12)
OpenJDK 64-Bit Server VM (build 25.191-b12, mixed mode)


In [6]:
from nltk.tag.stanford import StanfordPOSTagger as POS_Tag

#Check that paths are set properly for local host
jar = 'drive/My Drive/Colab Notebooks/NLP Project Testing/stanford-postagger.jar'
model = 'drive/My Drive/Colab Notebooks/NLP Project Testing/english-left3words-distsim.tagger'

st = POS_Tag(model_filename = model, path_to_jar = jar,  encoding='utf8')
#ner_tagger = StanfordNERTagger(model, jar, encoding='utf8')

/usr/local/lib/python3.6/dist-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


In [0]:
'''
Function "load" takes a .txt file as the parameter. It reads the .txt file, removes new line
indicators (\n), and returns the file as a string.
'''
def load(text):
    with open(text, encoding="ISO-8859-1") as f:
        my_text = f.read()
        lines = my_text.replace('\n',' ')
        return lines

In [0]:
def read_txts(path):
    txt_files = []

    for filename in os.walk(path):
      f = filename
      txt_files.append(f)
                    
    return txt_files

In [0]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [0]:
path = 'drive/My Drive/Colab Notebooks/txt'
test = read_txts(path) 

In [0]:
#shape files with flat list 
test2 = flatten(test)
test3 = test2[2:]
test4 = flatten(test3)

In [12]:
len(test4)

3036

In [0]:
#break up all files
txt_files = test4[991:999]

In [299]:
len(txt_files)

8

In [0]:
#Create list of unique authors
author_names = []
for i in range(len(txt_files)):
    string = txt_files[i]
    author_names.append(string[:string.index("_")])

In [0]:
#Load all txt files in list 
work_loaded = []
for filename in txt_files:
  full_name = os.path.join(path,filename)
  work_loaded.append(load(full_name))
  
#Takes 15 min to load all files

In [0]:
#Replace digits with '0'
digits = []
for word in work_loaded:
    temp_string = "".join(word)
    replaced_string = re.sub('\d', '0', temp_string)
    digits.append(replaced_string)

In [0]:
#Tokenize text
work_tokenized = []
for word in digits:
    work_tokenized.append(word_tokenize(word))
    
# 45 sec per folder

In [0]:
#POS tagging
final_tokens = []
for i in range((len(work_tokenized))):
            tag_tokens = st.tag(work_tokenized[i])
            final_tokens.append(tag_tokens)
      
# 5 min on 50 files

In [0]:
#put lists of tags and tokens together
token = []
tag = []
for i in range(len(final_tokens)):
    tokens, tags = zip(*final_tokens[i])
    token.append(tokens)
    tag.append(tags)

In [0]:
#Create dictionary for authors with their corresponding tokens
works = {'author': author_names,
         'tokens': token,
         'tags': tag}
author_works_df = pd.DataFrame.from_dict(works)
author_works_dict = author_works_df.groupby('author')[['tokens','tags']].apply(lambda g: g.values.tolist()).to_dict()

In [0]:
#author_works_dict[key][0][0] gives tokens
#author_works_dict[key][0][1] gives tags

In [0]:
#Write author/tokens dictionary to .txt file
#with open('authors_tokens_pos_test3.txt', 'w') as file:
#     file.write(json.dumps(author_works_dict))

In [0]:
author_works_df.to_csv('author_tokens_df_991_999.csv', encoding='utf-8', index=False)

In [0]:
'''
#Write author/tokens dictionary to .csv file
with open('authors_tokens_pos_Part1_50.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in author_works_dict.items():
        writer.writerow([key, value])
'''

"\n#Write author/tokens dictionary to .csv file\nwith open('authors_tokens_pos_Part1_50.csv', 'w') as csv_file:\n    writer = csv.writer(csv_file)\n    for key, value in author_works_dict.items():\n        writer.writerow([key, value])\n"

In [0]:
'''
for key, value in author_works_dict.items():
    temp_string = str(value)
    replaced_string = re.sub('\d', '0', temp_string)
    author_works_dict[key]= replaced_string
'''

In [0]:
'''
csv.field_size_limit(100000000)
with open('authors_tokens.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    mydict = dict(reader)
'''